## Preliminaries

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
from warnings import filterwarnings

# Important functionality for this notebook
from scipy import stats
import statsmodels.api as sm
import os
from pathlib import Path # For working with file paths
import re # For regex


In [2]:
# Set directory

os.chdir("C:/Users/emshe/Desktop/BRAINSTATION/CAPSTONE/GIT_REPO/DATA/ECONOMIC")

In [110]:
# Load geographic GEOJSON file

gdf = gpd.read_file("C:/Users/emshe/Desktop/BRAINSTATION/CAPSTONE/GIT_REPO/DATA/GEOGRAPHIC/PROCESSED/nbhds_with_zones.geojson")

In [22]:
# Define function to examine dataframes

def examine_df(name,df):
    """
    Check basic info about a dataframe df
    """
    
    print(f"\n\nNumber of records in the {name} is: {len(df)}\n")
    print(f"The columns in the {name} are: {df.columns}\n")
    print(f"\n Other info about {name}:")
    display(df.info())
    print(f"\n\nSample of records in the {name}:")
    display(df.head(5))

## Define base data processor class

In [49]:
# Define base data folder processor class

class BaseDataFolderProcessor:
    
    def __init__(self, folder_path, file_prefix, year_range = range(2016, 2025)):
        
        """
        Initialize data folder processor
        """
        
        self.folder_path = Path(folder_path)
        self.file_prefix = file_prefix
        self.year_range = year_range # Our default year range for this project is 2016 to 2024 so we make this argument optional
        self.vancouver_averages = {} 
        self.data_by_year = self.load_data()
        self.full_data = self.combine_yearly_data()
        
        
    def reset_data(self):
        pass
    
    def is_clean(self):
        pass

    def load_data(self): # Load CSV files into a dictionary keyed by year.

        """
        Load data from folder into dictionary
        """
        
        data_dict = {}
        for year in self.year_range:
            file_name = f"{self.file_prefix}_{year}.csv"
            file_path = self.folder_path / file_name
            if file_path.exists():
                try:
                    df = pd.read_csv(file_path, skiprows=2, encoding='latin1', na_values = '**',usecols=range(11))
                    df = self.clean_dataframe(df,file_name)
                    data_dict[year] = df
                except Exception as e:
                    print(f"Error loading {file_name}: {e}")
            else:
                print(f"Warning: {file_name} not found.")
        return data_dict
    
    def clean_dataframe(self, df, file_name):
        
        """
        Clean dataframe for specific year
        """

        # Determine the year from the filename
        match_year = re.search(r'_(\d{4})\.csv$', file_name)
        year = int(match_year.group(1)) if match_year else None
        
        match = re.search(r'by_(\w+)_\d{4}', file_name)
        if match:
            area_type = match.group(1).capitalize()
            df.columns.values[0] = area_type  # Rename first column
        else:
            df.columns.values[0] = 'area'  # Fallback name
    
        # Drop data quality columns based on their positions (C, E, G, I, K -> 2, 4, 6, 8, 10)
        dq_col_indices = [2, 4, 6, 8, 10]
        dq_cols = [df.columns[i] for i in dq_col_indices if i < len(df.columns)]
        df = df.drop(columns=dq_cols)

        
        # Store Vancouver-wide averages before dropping
        area_col = df.columns[0]
        vancouver_row = df[df[area_col].str.strip().str.lower() == 'vancouver']
    
        if not vancouver_row.empty and year is not None:
            self.vancouver_averages[year] = vancouver_row.squeeze()
        
        # Find the index of the row where area == 'Vancouver'
        vancouver_index = vancouver_row.index.min()
        
        # Drop that row and all rows after it in order to exclude unnecessary rows
        if pd.notnull(vancouver_index):
            df = df.loc[:vancouver_index - 1]

        # Standardize column names
        df.columns = (
            df.columns
            .str.lower()
            .str.replace(' ', '_')
            .str.replace('1', 'one')
            .str.replace('2', 'two')
            .str.replace('3', 'three')
            .str.replace('+', 'plus')
            .str.replace('neigh', 'nbhd')
            .str.replace('bachelor','studio')
        )
        
        return df

    def get_data_for_year(self, year):
        
        """
        Get dataframe for specific year
        """
        
        return self.data_by_year.get(year)


    def combine_yearly_data(self):
        
        """
        Combine the yearly data into a single long DataFrame with a 'year' column
        """
        
        combined_dfs = []
        for year in self.data_by_year.keys():
            df_copy = self.data_by_year[year].copy()
            if df_copy.empty:
                print(f"Warning: no valid data for year {year}, skipping.")
                continue
    
            df_copy['year'] = year
    
            # Reorder columns: move 'year' to second column
            cols = df_copy.columns.tolist()
            if 'year' in cols:
                cols.insert(1, cols.pop(cols.index('year')))
            df_copy = df_copy[cols]
            combined_dfs.append(df_copy)
    
        if not combined_dfs:
            raise ValueError("No valid dataframes to concatenate in combine_yearly_data.")
    
        self.full_data = pd.concat(combined_dfs, ignore_index=True)
        return self.full_data
        
    def save_processed_data(self):
        pass

In [72]:
# Define multifolder data folder processor class

class MultiFolderDataProcessor:
   
    def __init__(self, base_folder_path, folder_prefix_pairs, year_range=range(2016, 2025)):
        
        """
        Initialize multifolder data processor
        """
        
        self.base_folder_path = Path(base_folder_path)
        self.folder_prefix_pairs = folder_prefix_pairs
        self.year_range = year_range
        self.gdf = gdf # geographic dataframe for Vancouver

        self.processors = {}  # Store processors for later access
        self.data = self.load_all_data()
        self.completed_data = self.impute_neighborhood_rents()

    def load_all_data(self): 
        
        """
        Loads data from all subfolders and prefixes into a nested dictionary
        """
        
        all_data = {}
        for subfolder, prefix in self.folder_prefix_pairs:
            full_path = self.base_folder_path / subfolder
            processor = BaseDataFolderProcessor(full_path, prefix, self.year_range)
            all_data[subfolder] = processor.full_data
            self.processors[subfolder] = processor
        return all_data

    def get_data(self, subfolder, year):
        """
        Retrieve DataFrame for a specific subfolder and year
        """
        df = self.data.get(subfolder)
        if df is not None:
            return df[df['year'] == year].copy()
        return None

    # def impute_neighborhood_rents(self):
    #     """
    #     Replace missing rents in BY_NEIGHBORHOOD data using zone rents and then Vancouver-wide rents.
    #     """
    #     neighborhood_data = []

    #     for year in self.year_range:           
    #         nbhd_df = self.get_data("BY_NEIGHBORHOOD", year)
    #         zone_df = self.get_data("BY_ZONE", year)
    #         processor = self.processors.get("BY_NEIGHBORHOOD")
    #         vancouver_series = processor.vancouver_averages.get(year) if processor else None

    #         if nbhd_df is None or zone_df is None or vancouver_series is None:
    #             continue  # skip year if any component is missing

    #         # Merge neighborhood data with zone mapping
    #         merged = nbhd_df.merge(self.gdf, on='nbhd', how='left')

    #         # Merge with zone rent data
    #         merged = merged.merge(
    #             zone_df,
    #             on=['zone', 'year'],
    #             how='left',
    #             suffixes=('', '_zone')
    #         )

    #         # Fill NaNs in rent columns using zone values
    #         rent_columns = ['studio', 'one_bedroom', 'two_bedroom', 'three_bedroom_plus', 'total']
    #         for col in rent_columns:
    #             merged[col] = merged[col].fillna(merged[f'{col}_zone'])

    #         # Fill remaining NaNs using Vancouver-wide values
    #         if vancouver_series is not None:
    #             for col in rent_columns:
    #                 if col in vancouver_series.index:
    #                     merged[col] = merged[col].fillna(vancouver_series[col])

    #         # Drop extra zone columns but keep zone column
    #         merged = merged.drop(columns=[f'{col}_zone' for col in rent_columns])

    #         # Reorder columns
    #         final_cols = ['nbhd', 'zone', 'year'] + rent_columns
    #         merged = merged[final_cols]
    #         neighborhood_data.append(merged)

    #     # Combine all years
    #     return pd.concat(neighborhood_data, ignore_index=True)
    def impute_neighborhood_rents(self):
        neighborhood_data = []
    
        for year in self.year_range:           
            nbhd_df = self.get_data("BY_NEIGHBORHOOD", year)
            zone_df = self.get_data("BY_ZONE", year)
            processor = self.processors.get("BY_NEIGHBORHOOD")
            vancouver_series = processor.vancouver_averages.get(year) if processor else None
    
            if nbhd_df is None or zone_df is None or vancouver_series is None:
                continue
    
            # Clean 'nbhd' for join
            nbhd_df['nbhd'] = nbhd_df['nbhd'].str.strip().str.lower()
            gdf_clean = self.gdf.copy()
            gdf_clean['nbhd'] = gdf_clean['nbhd'].str.strip().str.lower()
    
            # Merge with zone info
            merged = nbhd_df.merge(gdf_clean, on='nbhd', how='left')
    
            # Debug zone join
            if merged['zone'].isnull().any():
                print(f"[DEBUG] Year {year}: Zone missing for neighborhoods:")
                print(merged.loc[merged['zone'].isnull(), 'nbhd'].unique())
    
            # Merge with zone-level rent data
            merged = merged.merge(
                zone_df,
                on=['zone', 'year'],
                how='left',
                suffixes=('', '_zone')
            )
    
            rent_columns = ['studio', 'one_bedroom', 'two_bedroom', 'three_bedroom_plus', 'total']
            for col in rent_columns:
                merged[col] = merged[col].fillna(merged[f'{col}_zone'])
    
            if vancouver_series is not None:
                for col in rent_columns:
                    if col in vancouver_series.index:
                        merged[col] = merged[col].fillna(vancouver_series[col])
    
            merged = merged.drop(columns=[f'{col}_zone' for col in rent_columns])
    
            final_cols = ['nbhd', 'zone', 'year'] + rent_columns
            merged = merged[final_cols]
            neighborhood_data.append(merged)
    
        return pd.concat(neighborhood_data, ignore_index=True)


In [111]:
# Test multifolder data processor

multi = MultiFolderDataProcessor(
    "AVG_RENTS",
    [("BY_NEIGHBORHOOD", "avg_rent_by_neigh"), ("BY_ZONE", "avg_rent_by_zone")]
)

# gdf must include columns: nbhd, zone
filled_nbhd_df = multi.completed_data
examine_df('filled df', filled_nbhd_df)



Number of records in the filled df is: 612

The columns in the filled df are: Index(['nbhd', 'zone', 'year', 'studio', 'one_bedroom', 'two_bedroom',
       'three_bedroom_plus', 'total'],
      dtype='object')


 Other info about filled df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   nbhd                612 non-null    object
 1   zone                612 non-null    object
 2   year                612 non-null    int64 
 3   studio              606 non-null    object
 4   one_bedroom         612 non-null    object
 5   two_bedroom         612 non-null    object
 6   three_bedroom_plus  563 non-null    object
 7   total               612 non-null    object
dtypes: int64(1), object(7)
memory usage: 38.4+ KB


None



Sample of records in the filled df:


,nbhd,zone,year,studio,one_bedroom,two_bedroom,three_bedroom_plus,total
0,ambleside,West Vancouver,2016,"1,120","1,461","2,337","3,670","1,723"
1,capital hill/brentwood,North Burnaby,2016,762,992,"1,242","1,377","1,059"
2,cedar cottage,Mount Pleasant/Renfrew Heights,2016,"1,085",962,"1,377","1,672","1,086"
3,central lonsdale,North Vancouver CY,2016,"1,068","1,271","1,493","1,818","1,312"
4,central park,Central Park/Metrotown,2016,"1,106","1,100","1,473","1,784","1,208"


## Testing function: test_sequence()

In [70]:
# Define function to streamline testing

def test_sequence(folder_name, file_name):
    
    """
    Test base data folder processor functionality
    """
    
    # Initialize avg rent by census tract class
    avg_rent_processor = BaseDataFolderProcessor(folder_name, file_name)

    # Extract data dictionary
    data_dict = avg_rent_processor.data_by_year
    
    # Extract 2020 specific data
    df_2020 = avg_rent_processor.get_data_for_year(2020)

    # Extract dataframe for all years
    avg_rent_df = avg_rent_processor.full_data

    # Print average vancouver rent dictionary
    # van_rent = avg_rent_processor.vancouver_averages
    # print(f''' Overall Vancouver rent:
    #         {van_rent}''')
    
    # Examine dataframe sample
    display(avg_rent_df.head(5))
    display(avg_rent_df.info())
    display(avg_rent_df['year'].value_counts())
    return avg_rent_df

In [71]:
# Run test sequence for rents by zone
avg_rent_by_zone_df = test_sequence("AVG_RENTS/BY_ZONE", "avg_rent_by_zone")

,zone,year,studio,one_bedroom,two_bedroom,three_bedroom_plus,total
0,West End/Stanley Park,2016,"1,050","1,366","2,109","3,158","1,418"
1,English Bay,2016,"1,133","1,441","2,110",NaN,"1,473"
2,Downtown,2016,"1,157","1,434","2,252","3,101","1,476"
3,South Granville/Oak,2016,"1,051","1,295","1,789","2,199","1,359"
4,Kitsilano/Point Grey,2016,"1,055","1,314","1,819","2,665","1,392"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   zone                225 non-null    object
 1   year                225 non-null    int64 
 2   studio              222 non-null    object
 3   one_bedroom         225 non-null    object
 4   two_bedroom         225 non-null    object
 5   three_bedroom_plus  199 non-null    object
 6   total               225 non-null    object
dtypes: int64(1), object(6)
memory usage: 12.4+ KB


None

year
2016    25
2017    25
2018    25
2019    25
2020    25
2021    25
2022    25
2023    25
2024    25
Name: count, dtype: int64

In [19]:
# Run test sequence for rents by neighborhood

avg_rent_by_nbhd_df = test_sequence("AVG_RENTS/BY_NEIGHBORHOOD", "avg_rent_by_neigh")

,nbhd,year,bachelor,one_bedroom,two_bedroom,three_bedroom_plus,total
0,Ambleside,2016,"1,120","1,461","2,337",NaN,"1,723"
1,Capital Hill/Brentwood,2016,762,992,"1,242","1,377","1,059"
2,Cedar Cottage,2016,"1,085",962,"1,377",NaN,"1,086"
3,Central Lonsdale,2016,"1,068","1,271","1,493",NaN,"1,312"
4,Central Park,2016,"1,106","1,100","1,473","1,784","1,208"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   nbhd                612 non-null    object
 1   year                612 non-null    int64 
 2   bachelor            445 non-null    object
 3   one_bedroom         579 non-null    object
 4   two_bedroom         574 non-null    object
 5   three_bedroom_plus  322 non-null    object
 6   total               583 non-null    object
dtypes: int64(1), object(6)
memory usage: 33.6+ KB


None

year
2016    68
2017    68
2018    68
2019    68
2020    68
2021    68
2022    68
2023    68
2024    68
Name: count, dtype: int64

In [104]:
# Get list of nbhds from rent df

rent_nbhds = list(avg_rent_by_nbhd_df['nbhd'].unique())
rent_nbhds.sort()

In [105]:
display(rent_nbhds)

['Ambleside',
 'Capital Hill/Brentwood',
 'Cedar Cottage',
 'Central Lonsdale',
 'Central Park',
 'Cloverdale/Fleetwood',
 'Collingwood',
 'Coquitlam East',
 'Coquitlam West',
 'Deer Lake',
 'Downtown',
 'Downtown Central',
 'Downtown Eastside/Strathcona',
 'Downtown North',
 'Dundrave/West Vancouver Remainder',
 'Edmonds',
 'English Bay',
 'Fraser View/Killarny',
 'Guildford',
 'Hastings/Sunrise/Grandview/Woodlands',
 'Kerrisdale',
 'Kitsilano/Point Grey North',
 'Kitsilano/Point Grey South',
 'Ladner',
 'Langley CY',
 'Langley DM',
 'Lonsdale East',
 'Lonsdale North',
 'Lonsdale West',
 'Maple Ridge',
 'Marpole Remainder',
 'Marpole South',
 'Metrotown',
 'Mount Pleasant',
 'New Westminster East',
 'New Westminster West',
 'Newton',
 'North Delta',
 'North False Creek',
 'North Vancouver DM East',
 'North Vancouver DM West',
 'Pitt Meadows',
 'Point Grey',
 'Port Coquitlam',
 'Port Moody',
 'Renfrew',
 'Richmond City Centre',
 'Richmond East',
 'Richmond South',
 'Richmond West',
 'R

In [106]:
gdf_nbhds = list(gdf['nbhd'].unique())
gdf_nbhds.sort()

In [109]:
examine_df('gdf',gdf)



Number of records in the gdf is: 68

The columns in the gdf are: Index(['nbhd', 'index_right', 'zone', 'geometry'], dtype='object')


 Other info about gdf:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   nbhd         68 non-null     object  
 1   index_right  64 non-null     float64 
 2   zone         64 non-null     object  
 3   geometry     68 non-null     geometry
dtypes: float64(1), geometry(1), object(2)
memory usage: 2.3+ KB


None



Sample of records in the gdf:


,nbhd,index_right,zone,geometry
0,West End/Stanley Park North,253.0,West End/Stanley Park,"POLYGON ((-13707903.9912 6324277.8928, -137077..."
1,West End/Stanley Park South,253.0,West End/Stanley Park,"POLYGON ((-13709954.8083 6328768.906, -1370897..."
2,English Bay,254.0,English Bay,"POLYGON ((-13708084.9552 6323746.4388, -137079..."
3,Downtown Central,255.0,Downtown,"POLYGON ((-13706769.2699 6325491.9126, -137066..."
4,North False Creek,255.0,Downtown,"POLYGON ((-13705341.8537 6321743.5686, -137053..."


In [107]:
display(gdf_nbhds)

['Ambleside',
 'Capital Hill/Brentwood',
 'Cedar Cottage',
 'Central Lonsdale',
 'Central Park',
 'Cloverdale/Fleetwood',
 'Collingwood',
 'Coquitlam East',
 'Coquitlam West',
 'Deer Lake',
 'Downtown',
 'Downtown Central',
 'Downtown Eastside/Strathcona',
 'Downtown North',
 'Dundrave/West Vancouver Remainder',
 'Edmonds',
 'English Bay',
 'Fraser View/Killarny',
 'Guildford',
 'Hastings/Sunrise/Grandview/Woodlands',
 'Kerrisdale',
 'Kitsilano/Point Grey North',
 'Kitsilano/Point Grey South',
 'Ladner',
 'Langley CY',
 'Langley DM',
 'Lonsdale East',
 'Lonsdale North',
 'Lonsdale West',
 'Maple Ridge',
 'Marpole Remainder',
 'Marpole South',
 'Metrotown',
 'Mount Pleasant',
 'New Westminster East',
 'New Westminster West',
 'Newton',
 'North Delta',
 'North False Creek',
 'North Vancouver DM East',
 'North Vancouver DM West',
 'Pitt Meadows',
 'Point Grey',
 'Port Coquitlam',
 'Port Moody',
 'Renfrew',
 'Richmond City Centre',
 'Richmond East',
 'Richmond South',
 'Richmond West',
 'R

## Test constructing economic dataframe with zones and neighborhoods

In [20]:
# Check average rent by nbhd dataframe
avg_rent_by_nbhd_df.head(10)

,nbhd,year,bachelor,one_bedroom,two_bedroom,three_bedroom_plus,total
0,Ambleside,2016,"1,120","1,461","2,337",NaN,"1,723"
1,Capital Hill/Brentwood,2016,762,992,"1,242","1,377","1,059"
2,Cedar Cottage,2016,"1,085",962,"1,377",NaN,"1,086"
3,Central Lonsdale,2016,"1,068","1,271","1,493",NaN,"1,312"
4,Central Park,2016,"1,106","1,100","1,473","1,784","1,208"
5,Cloverdale/Fleetwood,2016,NaN,807,"1,071",NaN,902
6,Collingwood,2016,976,"1,099","1,493",NaN,"1,249"
7,Coquitlam East,2016,NaN,"1,014","1,282",NaN,"1,204"
8,Coquitlam West,2016,783,892,"1,109","1,416",997
9,Deer Lake,2016,NaN,NaN,NaN,NaN,NaN


In [24]:
# Examine geographic dataframe gdf
examine_df('geographic data frame', gdf)



Number of records in the geographic data frame is: 68

The columns in the geographic data frame are: Index(['nbhd', 'index_right', 'zone', 'geometry'], dtype='object')


 Other info about geographic data frame:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   nbhd         68 non-null     object  
 1   index_right  64 non-null     float64 
 2   zone         64 non-null     object  
 3   geometry     68 non-null     geometry
dtypes: float64(1), geometry(1), object(2)
memory usage: 2.3+ KB


None



Sample of records in the geographic data frame:


,nbhd,index_right,zone,geometry
0,West End/Stanley Park North,253.0,West End/Stanley Park,"POLYGON ((-13707903.9912 6324277.8928, -137077..."
1,West End/Stanley Park South,253.0,West End/Stanley Park,"POLYGON ((-13709954.8083 6328768.906, -1370897..."
2,English Bay,254.0,English Bay,"POLYGON ((-13708084.9552 6323746.4388, -137079..."
3,Downtown Central,255.0,Downtown,"POLYGON ((-13706769.2699 6325491.9126, -137066..."
4,North False Creek,255.0,Downtown,"POLYGON ((-13705341.8537 6321743.5686, -137053..."


In [31]:
# Impute missing rent data

# Step 1: Merge neighborhood-to-zone info
nbhd_with_zone = avg_rent_by_nbhd_df.merge(gdf, on='nbhd', how='left')

# Step 2: Merge with zone-level rent data on zone + year
merged = nbhd_with_zone.merge(
    avg_rent_by_zone_df,
    on=['zone', 'year'],
    how='left',
    suffixes=('', '_zone')
)

display(merged.head(10))

# Step 3: Fill nulls in each rent column with corresponding zone values
rent_columns = ['bachelor', 'one_bedroom', 'two_bedroom', 'three_bedroom_plus', 'total']
for col in rent_columns:
    merged[col] = merged[col].fillna(merged[f"{col}_zone"])

# Step 4: Drop the temporary zone-based columns
merged = merged.drop(columns=[f"{col}_zone" for col in rent_columns])

# Optional: If you want the result in the same shape as avg_rent_by_nbhd_df
avg_rent_by_nbhd_df_filled = merged[avg_rent_by_nbhd_df.columns]


,nbhd,year,bachelor,one_bedroom,two_bedroom,three_bedroom_plus,total,index_right,zone,geometry,bachelor_zone,one_bedroom_zone,two_bedroom_zone,three_bedroom_plus_zone,total_zone
0,Ambleside,2016,"1,120","1,461","2,337",NaN,"1,723",270.0,West Vancouver,"POLYGON ((-13708220.2987 6331099.8864, -137081...","1,237","1,484","2,229","3,670","1,746"
1,Capital Hill/Brentwood,2016,762,992,"1,242","1,377","1,059",266.0,North Burnaby,"POLYGON ((-13688313.5153 6325092.3595, -136877...",820,"1,062","1,383","1,663","1,193"
2,Cedar Cottage,2016,"1,085",962,"1,377",NaN,"1,086",260.0,Mount Pleasant/Renfrew Heights,"POLYGON ((-13699643.2645 6318705.1709, -136996...",998,"1,103","1,458","1,672","1,148"
3,Central Lonsdale,2016,"1,068","1,271","1,493",NaN,"1,312",268.0,North Vancouver CY,"POLYGON ((-13700444.2632 6328120.2324, -137005...","1,001","1,176","1,439","1,818","1,249"
4,Central Park,2016,"1,106","1,100","1,473","1,784","1,208",264.0,Central Park/Metrotown,"POLYGON ((-13693045.1005 6316195.3582, -136928...",979,"1,034","1,372","1,741","1,125"
5,Cloverdale/Fleetwood,2016,NaN,807,"1,071",NaN,902,273.0,Surrey,"POLYGON ((-13656553.3403 6306265.4554, -136565...",728,855,"1,006","1,259",960
6,Collingwood,2016,976,"1,099","1,493",NaN,"1,249",262.0,Southeast Vancouver,"POLYGON ((-13698060.3777 6316470.0113, -136979...",958,"1,159","1,461","1,342","1,239"
7,Coquitlam East,2016,NaN,"1,014","1,282",NaN,"1,204",276.0,Tri-Cities,"POLYGON ((-13667322.2962 6334668.4205, -136670...",761,897,"1,130","1,475","1,042"
8,Coquitlam West,2016,783,892,"1,109","1,416",997,276.0,Tri-Cities,"POLYGON ((-13671782.674 6321398.1538, -1367221...",761,897,"1,130","1,475","1,042"
9,Deer Lake,2016,NaN,NaN,NaN,NaN,NaN,265.0,Southeast Burnaby,"POLYGON ((-13691923.8015 6318185.3615, -136919...",767,901,"1,099","1,265",970


In [33]:
examine_df('filled rent by nbhd df',avg_rent_by_nbhd_df_filled)



Number of records in the filled rent by nbhd df is: 612

The columns in the filled rent by nbhd df are: Index(['nbhd', 'year', 'bachelor', 'one_bedroom', 'two_bedroom',
       'three_bedroom_plus', 'total'],
      dtype='object')


 Other info about filled rent by nbhd df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   nbhd                612 non-null    object
 1   year                612 non-null    int64 
 2   bachelor            586 non-null    object
 3   one_bedroom         608 non-null    object
 4   two_bedroom         607 non-null    object
 5   three_bedroom_plus  551 non-null    object
 6   total               607 non-null    object
dtypes: int64(1), object(6)
memory usage: 33.6+ KB


None



Sample of records in the filled rent by nbhd df:


,nbhd,year,bachelor,one_bedroom,two_bedroom,three_bedroom_plus,total
0,Ambleside,2016,"1,120","1,461","2,337","3,670","1,723"
1,Capital Hill/Brentwood,2016,762,992,"1,242","1,377","1,059"
2,Cedar Cottage,2016,"1,085",962,"1,377","1,672","1,086"
3,Central Lonsdale,2016,"1,068","1,271","1,493","1,818","1,312"
4,Central Park,2016,"1,106","1,100","1,473","1,784","1,208"


## Working with tracts

In [20]:
# Run test sequence for rents by tract

avg_rent_by_tract_df = test_sequence("AVG_RENTS/BY_TRACT", "avg_rent_by_tract")

,Tract,Year,Bachelor,Bachelor DQ,1 Bedroom,1 Bedroom DQ,2 Bedroom,2 Bedroom DQ,3 Bedroom +,3 Bedroom + DQ,Total,Total DQ
0,0002.01,2016,NaN,NaN,NaN,NaN,"1,372",a,NaN,NaN,"1,153",a
1,0003.01,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0003.02,2016,NaN,NaN,837,a,970,b,NaN,NaN,861,a
3,0004.01,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0005.00,2016,833,a,963,a,"1,219",a,"1,393",a,995,a


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Tract           2551 non-null   object
 1   Year            2551 non-null   int64 
 2   Bachelor        681 non-null    object
 3   Bachelor DQ     681 non-null    object
 4   1 Bedroom       1099 non-null   object
 5   1 Bedroom DQ    1099 non-null   object
 6   2 Bedroom       1090 non-null   object
 7   2 Bedroom DQ    1090 non-null   object
 8   3 Bedroom +     224 non-null    object
 9   3 Bedroom + DQ  224 non-null    object
 10  Total           1202 non-null   object
 11  Total DQ        1202 non-null   object
dtypes: int64(1), object(11)
memory usage: 239.3+ KB


None

Year
2024    309
2023    307
2021    279
2022    279
2020    278
2018    277
2017    276
2019    274
2016    272
Name: count, dtype: int64

In [15]:
# Check and count tracts
tracts = avg_rent_by_tract_df['Tract'].unique()
print(f"Number of tracts is {len(tracts)}")
display(tracts[:10])

Number of tracts is 366


array(['0002.01', '0003.01', '0003.02', '0004.01', '0005.00', '0006.01',
       '0006.02', '0007.01', '0007.02', '0009.00'], dtype=object)

## Export files and information

In [15]:
# Extract list of zones
zones = avg_rent_by_zone_df['Zone'].unique()
display(zones)
print(f"Number of zones: {len(zones)}")

array(['West End/Stanley Park', 'English Bay', 'Downtown',
       'South Granville/Oak', 'Kitsilano/Point Grey',
       'Westside/Kerrisdale', 'Marpole', 'Mount Pleasant/Renfrew Heights',
       'East Hastings', 'Southeast Vancouver',
       'University Endowment Lands', 'Central Park/Metrotown',
       'Southeast Burnaby', 'North Burnaby', 'New Westminster',
       'North Vancouver CY', 'North Vancouver DM', 'West Vancouver',
       'Richmond', 'Delta', 'Surrey', 'White Rock',
       'Langley City and Langley DM', 'Tri-Cities',
       'Maple Ridge/Pitt Meadows'], dtype=object)

Number of zones: 25


In [11]:
# Check and count neighborhoods
neighs = avg_rent_by_neigh_df['Neigh'].unique()
print(f"Number of neighborhoods is {len(neighs)}")

Number of neighborhoods is 68


In [21]:
# Add 933 prefix to CMHC census tracts

# First, ensure the tract column is a string
avg_rent_by_tract_df['Tract'] = avg_rent_by_tract_df['Tract'].astype(str)

# Pad with '933' prefix to match StatsCan CTUID format
avg_rent_by_tract_df['Tract'] = '933' + avg_rent_by_tract_df['Tract']

In [17]:
# Export average rents by neighborhood to csv
avg_rent_by_neigh_df.to_csv("C:/Users/emshe/Desktop/BRAINSTATION/CAPSTONE/GIT_REPO/DATA/ECONOMIC/PROCESSED_DATAFRAMES/avg_rent_by_neigh.csv", index=False)

In [23]:
# Export average rents by tract to csv
avg_rent_by_tract_df.to_csv("C:/Users/emshe/Desktop/BRAINSTATION/CAPSTONE/GIT_REPO/DATA/ECONOMIC/PROCESSED_DATAFRAMES/avg_rent_by_tract.csv", index=False)

## Miscellaneous

In [8]:
# Define subfolders and prefixes
folder_prefixes = [
    ("BY_TRACT", "avg_rent_by_tract"),
    ("BY_NEIGHBORHOOD", "avg_rent_by_neigh")
]

# Initialize
multi_proc = MultiFolderDataProcessor("AVG_RENTS", folder_prefixes)

# Get 2020 data from BY_NEIGHBORHOOD
df_neigh_2020 = multi_proc.get_data("BY_NEIGHBORHOOD", 2020)

# Get 2017 data from BY_TRACT
df_tract_2017 = multi_proc.get_data("BY_TRACT", 2017)


display(df_neigh_2020.tail(5))
display(df_tract_2017.tail(5))

,Neigh,Bachelor,Bachelor DQ,1 Bedroom,1 Bedroom DQ,2 Bedroom,2 Bedroom DQ,3 Bedroom +,3 Bedroom + DQ,Total,Total DQ
63,West End/Stanley Park South,"1,202",a,"1,493",a,"2,196",a,NaN,NaN,"1,499",a
64,Westside Heights,"1,020",a,"1,253",a,"1,482",a,"1,742",a,"1,284",a
65,Westside/Kerrisdale Remainder,NaN,NaN,"1,609",a,"2,291",a,"2,999",a,"1,919",a
66,Whalley,715,b,"1,185",a,"1,207",a,"1,535",a,"1,242",a
67,White Rock,946,b,"1,091",a,"1,422",a,NaN,NaN,"1,191",a


,Tract,Bachelor,Bachelor DQ,1 Bedroom,1 Bedroom DQ,2 Bedroom,2 Bedroom DQ,3 Bedroom +,3 Bedroom + DQ,Total,Total DQ
271,0504.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,0504.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,0505.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,0506.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,0506.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Define subfolders and prefixes
folder_prefixes = [
    ("BY_TRACT", "avg_rent_by_tract"),
    ("BY_NEIGHBORHOOD", "avg_rent_by_neigh")
]

# Initialize
multi_proc = MultiFolderDataProcessor("AVG_RENTS", folder_prefixes)

# Get 2020 data from BY_NEIGHBORHOOD
df_neigh_2016 = multi_proc.get_data("BY_NEIGHBORHOOD", 2020)

# Get 2017 data from BY_TRACT
df_tract_2020 = multi_proc.get_data("BY_TRACT", 2020)


display(df_neigh_2016.tail(5))
display(df_tract_2020.tail(5))

,Neigh,Bachelor,Bachelor DQ,1 Bedroom,1 Bedroom DQ,2 Bedroom,2 Bedroom DQ,3 Bedroom +,3 Bedroom + DQ,Total,Total DQ
63,West End/Stanley Park South,"1,202",a,"1,493",a,"2,196",a,NaN,NaN,"1,499",a
64,Westside Heights,"1,020",a,"1,253",a,"1,482",a,"1,742",a,"1,284",a
65,Westside/Kerrisdale Remainder,NaN,NaN,"1,609",a,"2,291",a,"2,999",a,"1,919",a
66,Whalley,715,b,"1,185",a,"1,207",a,"1,535",a,"1,242",a
67,White Rock,946,b,"1,091",a,"1,422",a,NaN,NaN,"1,191",a


,Tract,Bachelor,Bachelor DQ,1 Bedroom,1 Bedroom DQ,2 Bedroom,2 Bedroom DQ,3 Bedroom +,3 Bedroom + DQ,Total,Total DQ
273,0504.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,0504.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,0505.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276,0506.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,0506.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
